In [152]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, root
import statsmodels.formula.api as sm
from Fixed_Income_Toolbox import *

In [153]:
maturity = np.arange(1,7)
spot = [0.05, 0.055, 0.057, 0.059, 0.06, 0.061]
price = []
for i in range(len(maturity)):
    bond = ZeroCouponBond(100, maturity[i])
    price.append(bond.get_price(spot[i]))
price

[95.23809523809524,
 89.84524157139327,
 84.6788669093383,
 79.50897588580243,
 74.7258172866057,
 70.09833403416522]

In [154]:
def ho_and_lee_calibrate(level, spot, vol, price, k=1, fv=100):
    calibrate_lv = [s + vol for s in spot[level - 1]] + [spot[level - 1][-1] - vol] # construct the last level of spot
    def function(x, *args):
        calibrate_lv, spot, price, fv = args
        price_tree = [fv / (1 + spot + x) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j])
        return price_tree[0] - price
    args = (calibrate_lv, spot, price, fv)
    m = fsolve(function, 0.01, args=args)
    return [s + m[0] for s in calibrate_lv]

In [155]:
# test calibration using the example in class
spot_dict = {1:[0.05263], 2:[0.07593, 0.03593]}
ho_and_lee_calibrate(3, spot_dict, 0.02, 85)

[0.09996345790671082, 0.05996345790671081, 0.01996345790671081]

In [156]:
# construct the ho and lee tree
vol = 0.015
spot_dict = {1:[spot[0]]}
for lv in range(2, len(price)+1):
    spot_dict[lv] = ho_and_lee_calibrate(lv, spot_dict, vol, price[lv-1])
spot_dict

{1: [0.05],
 2: [0.07523602642117286, 0.04523602642117285],
 3: [0.09164759479639102, 0.061647594796391016, 0.03164759479639102],
 4: [0.11129248303472124,
  0.08129248303472124,
  0.05129248303472124,
  0.021292483034721237],
 5: [0.1261246338504627,
  0.09612463385046267,
  0.06612463385046267,
  0.03612463385046268,
  0.006124633850462678],
 6: [0.14418404774940974,
  0.11418404774940971,
  0.08418404774940971,
  0.05418404774940973,
  0.024184047749409722,
  -0.005815952250590278]}

In [157]:
def get_payoff_tree(spot_dict, cf_dict, k=1):
    sorted_lv = sorted(spot_dict.keys(), reverse=True)
    last_period = sorted_lv[0]
    payoff_dict = {}
    for lv in sorted_lv:
        if lv == last_period:
            payoff_dict[lv] = np.divide(cf_dict[lv], [s / k + 1 for s in spot_dict[lv]]).tolist()
        else:
            payoff_dict[lv] = np.zeros(lv).tolist()
            for i in range(lv):
                payoff_dict[lv][i] = (0.5 * (payoff_dict[lv+1][i] + payoff_dict[lv+1][i+1]) + cf_dict[lv][i]) / (1 + spot_dict[lv][i] / k)
    return payoff_dict

In [161]:
# part a
bond = ZeroCouponBond(100, 6)
z_ls = [bond.get_discount_function(0.055, t) for t in range(1,7)]
fixed_pay = 100 / sum(z_ls)
fixed_cf = {i:[fixed_pay]*i for i in range(1, 7)}
fixed_payoff = get_payoff_tree(spot_dict, fixed_cf)
duration_a = - (fixed_payoff[2][1] - fixed_payoff[2][0]) / (spot_dict[2][1] - spot_dict[2][0]) / fixed_payoff[1][0]
duration_a

2.2930854075483995

In [162]:
# part b
# first get payoff at each node
fixed_cf = {i:[fixed_pay]*i for i in range(1, 7)}
fixed_payoff = get_payoff_tree(spot_dict, fixed_cf)
# then construct a new cash flow tree that computes extra payoff if 5.5% interest rate is quoted
cf_dict = {i:[0]*i for i in range(1, 7)}
for key in spot_dict:
    for i in range(key):
        cf_dict[key][i] = fixed_pay * (spot_dict[key][i] - 0.055)
# get rid of the nodes that have negative cashflow
# then add one time payment from the initial payoff tree
option_payoff = get_payoff_tree(spot_dict, cf_dict)
option_cf = {i:[0]*i for i in range(1, 7)}
threshold = 6
for lv in range(1, 7):
    for i in range(lv):
        if option_payoff[lv][i] >=0 and i < threshold:
            option_cf[lv][i] = fixed_pay
        elif option_payoff[lv][i] < 0 and i < threshold:
            option_cf[lv][i] = fixed_payoff[lv][i]
            threshold = i
# using the new cash flow tree, get the payoff again
final_payoff = get_payoff_tree(spot_dict, option_cf)
print(final_payoff)
duration_b = - (final_payoff[2][1] - final_payoff[2][0]) / ((spot_dict[2][1] - spot_dict[2][0]) * final_payoff[1][0])
duration_b

{6: [17.495345090340024, 0.0, 0.0, 0.0, 0.0, 0.0], 5: [25.543857618185967, 0.0, 0.0, 0.0, 0.0], 4: [29.506024806168433, 0.0, 0.0, 0.0], 3: [31.851769134165988, 0.0, 0.0], 2: [33.428734199835795, 83.46154377869296], 1: [74.72669881095466]}


22.31813188368427

In [163]:
# data from PS1
df = pd.read_excel('HW1_data.xls')
bond = ZeroCouponBond(100, df['Maturity'])
df['Spot'] = bond.get_spot(df['Price'], k=2)
# estimate term structure
new_df = pd.DataFrame(df['Maturity'].apply(lambda x: x**i) for i in range(1,6)).T
new_df.columns = ['M1','M2','M3','M4','M5']
df['Discount Function'] = bond.get_discount_function(df['Spot'], df['Maturity'], k=2)
new_df['logZ'] = np.log(df['Discount Function'])
rls = sm.ols(formula="logZ ~ %s + 0" % "+".join(new_df.loc[:,'M1':'M5'].columns.tolist()),data=new_df).fit()
# predict new maturity
predict_series = pd.Series(np.linspace(0.5, 10, 20))
predict_df = pd.DataFrame(predict_series.apply(lambda x: x**i) for i in range(1,6)).T
predict_df['Estimated Z'] = np.exp(np.dot(predict_df, rls.params))
predict_df['Price'] = 100 * predict_df['Estimated Z']
predict_df['Spot'] = (predict_df['Estimated Z']**(- 1 / (2 * predict_df.loc[:,0])) - 1) * 2
predict_df.head()

,0,1,2,3,4,Estimated Z,Price,Spot
0,0.5,0.25,0.125,0.0625,0.03125,0.983559,98.355909,0.033431
1,1.0,1.00,1.000,1.0000,1.00000,0.966855,96.685521,0.033992
2,1.5,2.25,3.375,5.0625,7.59375,0.949903,94.990280,0.034559
3,2.0,4.00,8.000,16.0000,32.00000,0.932721,93.272057,0.035130
4,2.5,6.25,15.625,39.0625,97.65625,0.915331,91.533093,0.035703


In [210]:
# BDT calibration
def bdt_calibrate(level, spot, sigma, prop_vol, price, k=1, fv=100):
    def calibrate(m, level, spot, sigma):
        calibrate_lv = [spot[level-1][0] * np.exp(m + sigma[level-1])]
        for i in range(1, level):
            calibrate_lv += [calibrate_lv[i-1] * np.exp(- 2 * sigma[level-1])]
        return calibrate_lv
        
    def function(params, *args):
        level, spot, sigma, price, k, fv = args
        m, sigma[level-1] = params
        calibrate_lv = calibrate(m, level, spot, sigma)
        price_tree = [fv / (1 + spot / k) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                if i == 1:
                    bond = ZeroCouponBond(fv, level-1)
                    spot_u = bond.get_spot(price_tree[j], k=k)
                    spot_d = bond.get_spot(price_tree[j+1], k=k)
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j] / k)
        return [price_tree[0] - price, 0.5 * np.log(spot_u / spot_d) - prop_vol]
    
    args = (level, spot, sigma, price, k, fv)
    r = fsolve(function, [0.01, 0.1], args=args)
    return calibrate(r[0], level, spot, sigma)

In [211]:
# test in class example
spot_dict = {1:[0.05263], 2:[0.0639, 0.04734]}
sigma = {1:0.15, 2:0.13}
prop_vol = 0.15
price = 90
bdt_calibrate(2, spot_dict, sigma, prop_vol, price)

[0.06390340424825898, 0.047340806230699646]

In [213]:
# calibrate 10 years model
prop_vol = 0.15 * np.sqrt(0.5)
spot_dict = {1:[predict_df.loc[0, 'Spot']]}
sigma = {1:prop_vol}
for lv in range(2, len(predict_df.index)+1):
    spot_dict[lv] = bdt_calibrate(lv, spot_dict, sigma, prop_vol, predict_df.loc[lv-1, 'Price'], k=2)
spot_dict

{1: [0.033431460302576266],
 2: [0.03822701005487272, 0.030892244589409704],
 3: [0.043698678889266075, 0.03531151825157295, 0.02853411940417843],
 4: [0.049933976824511665,
  0.040345359359113375,
  0.03259800491229783,
  0.026338343272735334],
 5: [0.057031985063596954,
  0.04607277246779216,
  0.037219471854291974,
  0.03006741315775687,
  0.024289687331901254],
 6: [0.06510518733327798,
  0.05258327027323487,
  0.042469738985218435,
  0.034301379889463884,
  0.027704070955812253,
  0.02237564640250617],
 7: [0.07428165153039433,
  0.059978556373546064,
  0.04842955360493916,
  0.039104336686038334,
  0.03157471076708605,
  0.02549493085714689,
  0.020585825922695748],
 8: [0.0847076346282119,
  0.06837453156829666,
  0.055190734432654806,
  0.04454900234560701,
  0.03595918101815762,
  0.02902562642065848,
  0.023428981563462556,
  0.018911467030747222],
 9: [0.09655069999289533,
  0.07790369125620596,
  0.06285801254458896,
  0.050718132572966486,
  0.0409228492527392,
  0.0330193